# 使用 Milvus 和 DeepSeek 构建 RAG

DeepSeek 帮助开发者使用高性能语言模型构建和扩展 AI 应用。它提供高效的推理、灵活的 API 以及先进的专家混合 (MoE) 架构，用于强大的推理和检索任务。

在本教程中，我们将展示如何使用 Milvus 和 DeepSeek 构建一个检索增强生成 (RAG) 管道。

## 准备工作

### 依赖与环境

In [1]:
!pip install "pymilvus[model]==2.5.10" openai==1.82.0 requests==2.32.3 tqdm==4.67.1 torch==2.7.0

---

In [110]:
import os

# 从环境变量获取 DeepSeek API Key
api_key = os.getenv("DEEPSEEK_API_KEY")

### 准备数据

我们使用 Milvus 文档 2.4.x 中的 FAQ 页面作为我们 RAG 中的私有知识库，这是一个简单 RAG 管道的良好数据源。

下载 zip 文件并将文档解压到 `milvus_docs` 文件夹。

**建议在命令行执行下面命令**

In [18]:
#!wget https://github.com/milvus-io/milvus-docs/releases/download/v2.4.6-preview/milvus_docs_2.4.x_en.zip
#!unzip -q milvus_docs_2.4.x_en.zip -d milvus_docs

我们从 `milvus_docs/en/faq` 文件夹加载所有 markdown 文件。对于每个文档，我们简单地使用 "# " 来分割文件中的内容，这样可以大致分离出 markdown 文件中每个主要部分的内容。

In [75]:
from glob import glob

text_lines = []
text_lines_mfd = []

for file_path in glob("milvus_docs/en/faq/*.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines += file_text.split("# ")

for file_path in glob("mfd.md", recursive=True):
    with open(file_path, "r") as file:
        file_text = file.read()

    text_lines_mfd += file_text.split("# ")

In [76]:
len(text_lines)

72

In [77]:
len(text_lines_mfd)

30

### 准备 LLM 和 Embedding 模型

DeepSeek 支持 OpenAI 风格的 API，您可以使用相同的 API 进行微小调整来调用 LLM。

In [78]:
from openai import OpenAI

deepseek_client = OpenAI(
    api_key=api_key,
    base_url="https://api.deepseek.com/v1",  # DeepSeek API 的基地址
)

定义一个 embedding 模型，使用 `milvus_model` 来生成文本嵌入。我们以 `DefaultEmbeddingFunction` 模型为例，这是一个预训练的轻量级嵌入模型。

In [79]:
from pymilvus import model as milvus_model

embedding_model = milvus_model.DefaultEmbeddingFunction()

生成一个测试嵌入并打印其维度和前几个元素。

In [80]:
test_embedding = embedding_model.encode_queries(["This is a test"])[0]
embedding_dim = len(test_embedding)
print(embedding_dim)
print(test_embedding[:10])

768
[-0.04836059  0.07163021 -0.01130063 -0.03789341 -0.03320651 -0.01318453
 -0.03041721 -0.02269495 -0.02317858 -0.00426026]


In [81]:
test_embedding_0 = embedding_model.encode_queries(["That is a test"])[0]
print(test_embedding_0[:10])

[-0.02752976  0.0608853   0.00388525 -0.00215193 -0.02774976 -0.0118618
 -0.04020916 -0.06023417 -0.03813156  0.0100272 ]


## 将数据加载到 Milvus

### 创建 Collection

In [82]:
from pymilvus import MilvusClient

milvus_client = MilvusClient(uri="./milvus_demo.db")

collection_name = "my_rag_collection"
collection_name_mfd = "mfd_rag_collection"

关于 `MilvusClient` 的参数：

*   将 `uri` 设置为本地文件，例如 `./milvus.db`，是最方便的方法，因为它会自动利用 Milvus Lite 将所有数据存储在此文件中。
*   如果您有大规模数据，可以在 Docker 或 Kubernetes 上设置性能更高的 Milvus 服务器。在此设置中，请使用服务器 URI，例如 `http://localhost:19530`，作为您的 `uri`。
*   如果您想使用 Zilliz Cloud（Milvus 的完全托管云服务），请调整 `uri` 和 `token`，它们对应 Zilliz Cloud 中的 Public Endpoint 和 Api key。

检查 collection 是否已存在，如果存在则删除它。

In [83]:
for collection in [collection_name, collection_name_mfd]:
    if milvus_client.has_collection(collection):
        milvus_client.drop_collection(collection)

创建一个具有指定参数的新 collection。

如果我们不指定任何字段信息，Milvus 将自动创建一个默认的 `id` 字段作为主键，以及一个 `vector` 字段来存储向量数据。一个保留的 JSON 字段用于存储非 schema 定义的字段及其值。

`metric_type` (距离度量类型):
     作用：定义如何计算向量之间的相似程度。
     例如：`IP` (内积) - 值越大通常越相似；`L2` (欧氏距离) - 值越小越相似；`COSINE` (余弦相似度) - 通常转换为距离，值越小越相似。
     选择依据：根据你的嵌入模型的特性和期望的相似性定义来选择。

 `consistency_level` (一致性级别):
     作用：定义数据写入后，读取操作能多快看到这些新数据。
     例如：
         `Strong` (强一致性): 总是读到最新数据，可能稍慢。
         `Bounded` (有界过期): 可能读到几秒内旧数据，性能较好 (默认)。
         `Session` (会话一致性): 自己写入的自己能立刻读到。
         `Eventually` (最终一致性): 最终会读到新数据，但没时间保证，性能最好。
     选择依据：在数据实时性要求和系统性能之间做权衡。

简单来说：
 `metric_type`：怎么算相似。
 `consistency_level`：新数据多久能被读到。

In [84]:
for collection in [collection_name, collection_name_mfd]:
    milvus_client.create_collection(
        collection_name=collection,
        dimension=embedding_dim,
        metric_type="IP",  # 内积距离
        consistency_level="Strong",  # 支持的值为 (`"Strong"`, `"Session"`, `"Bounded"`, `"Eventually"`)。更多详情请参见 https://milvus.io/docs/consistency.md#Consistency-Level。
    )

# List all collections
collections = milvus_client.list_collections()

print("All collections:", collections)

All collections: ['mfd_rag_collection', 'my_rag_collection']


### 插入数据

遍历文本行，创建嵌入，然后将数据插入 Milvus。

这里有一个新字段 `text`，它是在 collection schema 中未定义的字段。它将自动添加到保留的 JSON 动态字段中，该字段在高级别上可以被视为普通字段。

In [86]:
from tqdm import tqdm

data = []

doc_embeddings = embedding_model.encode_documents(text_lines)

for i, line in enumerate(tqdm(text_lines, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
milvus_client.insert(collection_name=collection_name, data=data)

data = []
doc_embeddings = embedding_model.encode_documents(text_lines_mfd)
for i, line in enumerate(tqdm(text_lines_mfd, desc="Creating embeddings")):
    data.append({"id": i, "vector": doc_embeddings[i], "text": line})
milvus_client.insert(collection_name=collection_name_mfd, data=data)

Creating embeddings: 100%|██████████| 30/30 [00:00<00:00, 117817.53it/s]


{'insert_count': 30, 'ids': [0, 1, 2, 3, 4, 5, 6, 7, 8, 9, 10, 11, 12, 13, 14, 15, 16, 17, 18, 19, 20, 21, 22, 23, 24, 25, 26, 27, 28, 29], 'cost': 0}

## 构建 RAG

### 检索查询数据

我们指定一个关于 Milvus 的常见问题。

In [88]:
question = "What is milvus? How much does it cost?"
print(question)
question_mfd = "登记机构应该履行什么职责？"
print(question_mfd)

What is milvus? How much does it cost?
登记机构应该履行什么职责？


在 collection 中搜索该问题，并检索语义上最匹配的前3个结果。

In [93]:
search_res = milvus_client.search(
    collection_name=collection_name,
    data=embedding_model.encode_queries(
        [question]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

search_res_mfd = milvus_client.search(
    collection_name=collection_name_mfd,
    data=embedding_model.encode_queries(
        [question_mfd]
    ),  # 将问题转换为嵌入向量
    limit=3,  # 返回前3个结果
    search_params={"metric_type": "IP", "params": {}},  # 内积距离
    output_fields=["text"],  # 返回 text 字段
)

让我们看一下查询的搜索结果

In [94]:
import json

retrieved_lines_with_distances = [
    (res["entity"]["text"], res["distance"]) for res in search_res[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))

retrieved_lines_with_distances_mfd = [
    (res["entity"]["text"], res["distance"]) for res in search_res_mfd[0]
]
print(json.dumps(retrieved_lines_with_distances, indent=4))
print(json.dumps(retrieved_lines_with_distances_mfd, indent=4))

[
    [
        "How much does Milvus cost?\n\nMilvus is a 100% free open-source project.\n\nPlease adhere to [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0) when using Milvus for production or distribution purposes.\n\nZilliz, the company behind Milvus, also offers a fully managed cloud version of the platform for those that don't want to build and maintain their own distributed instance. [Zilliz Cloud](https://zilliz.com/cloud) automatically maintains data reliability and allows users to pay only for what they use.\n\n###",
        0.6218432784080505
    ],
    [
        "Does Milvus support Apple M1 CPU?\n\nCurrent Milvus release does not support Apple M1 CPU.\n\n###",
        0.4678838849067688
    ]
]
[
    [
        "How much does Milvus cost?\n\nMilvus is a 100% free open-source project.\n\nPlease adhere to [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0) when using Milvus for production or distribution purposes.\n\nZilliz, the company behind Mil

### 使用 LLM 获取 RAG 响应

将检索到的文档转换为字符串格式。

In [96]:
context = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances]
)

context_mfd = "\n".join(
    [line_with_distance[0] for line_with_distance in retrieved_lines_with_distances_mfd]
)

In [97]:
print(context)
print(context_mfd)

How much does Milvus cost?

Milvus is a 100% free open-source project.

Please adhere to [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0) when using Milvus for production or distribution purposes.

Zilliz, the company behind Milvus, also offers a fully managed cloud version of the platform for those that don't want to build and maintain their own distributed instance. [Zilliz Cloud](https://zilliz.com/cloud) automatically maintains data reliability and allows users to pay only for what they use.

###
Does Milvus support Apple M1 CPU?

Current Milvus release does not support Apple M1 CPU.

###
二、权利质权

**第四百四十九条** 可以出质的权利包括：
（一）汇票、本票、支票；
（二）债券、存款单；
（三）仓单、提单；
（四）可以转让的基金份额、股权；
（五）可以转让的注册商标专用权、专利权、著作权等知识产权中的财产权；
（六）应收账款；
（七）法律、行政法规规定可以出质的其他财产权利。

**第四百五十条** 以汇票、本票、支票、债券、存款单、仓单、提单出质的，当事人应当订立书面合同。质权自权利凭证交付之日起设立。

**第四百五十一条** 以记名股票出质的，当事人应当订立书面合同。质权自股票交付之日起设立。
以未上市公司股权出质的，适用公司法有关股权转让的规定。

**第四百五十二条** 以可以转让的基金份额、股权出质的，当事人应当订立书面合同。质权自基金份额、股权登记于证券登记结算机构或者公司章程载明的股权登记簿时设立。
以未上市公司股权出

In [98]:
print(question)
print(question_mfd)

What is milvus? How much does it cost?
登记机构应该履行什么职责？


为语言模型定义系统和用户提示。此提示是使用从 Milvus 检索到的文档组装而成的。

In [103]:
SYSTEM_PROMPT = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
{context}
</context>
<question>
{question}
</question>
<translated>
</translated>
"""

SYSTEM_PROMPT_MFD = """
Human: 你是一个 AI 助手。你能够从提供的上下文段落片段中找到问题的答案。
"""
USER_PROMPT_MFD = f"""
请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
{context_mfd}
</context>
<question>
{question_mfd}
</question>
"""

In [104]:
print(USER_PROMPT)
print(USER_PROMPT_MFD)


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。最后追加原始回答的中文翻译，并用 <translated>和</translated> 标签标注。
<context>
How much does Milvus cost?

Milvus is a 100% free open-source project.

Please adhere to [Apache License 2.0](http://www.apache.org/licenses/LICENSE-2.0) when using Milvus for production or distribution purposes.

Zilliz, the company behind Milvus, also offers a fully managed cloud version of the platform for those that don't want to build and maintain their own distributed instance. [Zilliz Cloud](https://zilliz.com/cloud) automatically maintains data reliability and allows users to pay only for what they use.

###
Does Milvus support Apple M1 CPU?

Current Milvus release does not support Apple M1 CPU.

###
</context>
<question>
What is milvus? How much does it cost?
</question>
<translated>
</translated>


请使用以下用 <context> 标签括起来的信息片段来回答用 <question> 标签括起来的问题。
<context>
二、权利质权

**第四百四十九条** 可以出质的权利包括：
（一）汇票、本票、支票；
（二）债券、存款单；
（三）仓单、提单；
（四）可以转让的基金份额、股权；
（五）可以转让的注册商标专用权、专利权、著作权等

使用 DeepSeek 提供的 `deepseek-chat` 模型根据提示生成响应。

In [105]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT},
    ],
)
print(response.choices[0].message.content)

Milvus is a 100% free open-source project. It is a vector database designed for AI applications, allowing efficient similarity search and analysis of unstructured data. The open-source version has no cost, but users must adhere to the Apache License 2.0 when using it for production or distribution. 

For those who prefer a managed service, Zilliz (the company behind Milvus) offers Zilliz Cloud, a paid cloud version where users pay only for what they use.

<translated>
Milvus是一个100%免费的开源项目。它是一个专为AI应用设计的向量数据库，支持对非结构化数据进行高效的相似性搜索和分析。开源版本完全免费，但在生产或分发使用时需遵守Apache 2.0许可证。

对于需要托管服务的用户，Milvus背后的公司Zilliz提供了付费的Zilliz Cloud云服务版本，采用按用量付费的模式。
</translated>


In [106]:
response = deepseek_client.chat.completions.create(
    model="deepseek-chat",
    messages=[
        {"role": "system", "content": SYSTEM_PROMPT_MFD},
        {"role": "user", "content": USER_PROMPT_MFD},
    ],
)
print(response.choices[0].message.content)

根据提供的上下文，登记机构在地役权相关事项中应履行的职责主要包括以下内容：

1. **办理地役权设立登记**（依据第三百九十条）：
   - 设立地役权时，登记机构应依法办理登记手续
   - 未经登记的地役权不得对抗善意第三人

2. **办理地役权注销登记**（依据第三百九十三条）：
   - 当地役权期限届满或地役权人放弃地役权时
   - 登记机构应当依法办理注销登记手续

需要注意的是，上下文未涉及其他类型权利（如权利质权）中登记机构的具体职责，上述回答仅基于地役权相关条款（第三百八十八条至第三百九十八条）的信息。
